# 20대 의안활동보고서 분석 

작성자: 박하람

In [1]:
import sys
sys.path.append("/usr/local/lib/python3.8/site-packages")

In [2]:
from glob import glob
import pandas as pd
import numpy as np
from ast import literal_eval
import re
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [3]:
# 한글폰트 설정
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

Unable to locate an executable at "/Library/Java/JavaVirtualMachines/jdk-14.0.1.jdk/Contents/Home/bin/apt" (-1)


In [4]:
files = glob('final-data/final-20th-*.csv')

for index, file in enumerate(files): 
    df = pd.read_csv(file, index_col=0, encoding='utf-8')
    print(index, len(df))
    if index != 0: 
        df_com = pd.concat([df_com, df])
    else: 
        df_com = df

0 7221
1 6001
2 12000


## 전처리

###  불필요한 칼럼 제거

In [5]:
df_com = df_com.drop(['index','심사진행상태','문서'],1)

bill_df = df_com

In [6]:
bill_df.head(1)

,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,제안회기,제안이유,소관위원회,제안자
의안번호,,,,,,,,,,
2007178,처,국가표준기본법 일부개정법률안(문미옥의원 등 28인),의원,2017-06-02,2017-11-24,수정가결,제20대 (2016~2020) 제351회,['제안이유 및 주요내용4차 산업혁명 시대의 도래와 함께 전세계적으로 참조표준 데이...,산업통상자원중소벤처기업위원회,"['문미옥(더불어민주당/文美玉)', '강창일(더불어민주당/姜昌一)', '기동민(더불..."


In [7]:
bill_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25222 entries, 2007178 to 2013116
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   처리여부    25222 non-null  object
 1   의안명     25222 non-null  object
 2   제안자구분   25222 non-null  object
 3   제안일자    25222 non-null  object
 4   의결일자    24998 non-null  object
 5   의결결과    24998 non-null  object
 6   제안회기    25222 non-null  object
 7   제안이유    24665 non-null  object
 8   소관위원회   25109 non-null  object
 9   제안자     23591 non-null  object
dtypes: object(10)
memory usage: 2.1+ MB


In [8]:
bill_df.isnull().sum()

처리여부        0
의안명         0
제안자구분       0
제안일자        0
의결일자      224
의결결과      224
제안회기        0
제안이유      557
소관위원회     113
제안자      1631
dtype: int64

In [9]:
bill_df[bill_df['제안이유'].isnull()].head()

,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,제안회기,제안이유,소관위원회,제안자
의안번호,,,,,,,,,,
ZZ23959,계,「인공지능산업 진흥에 관한 법률안」에 대한 의견표명 결정문,기타,2020-05-28,NaN,NaN,제20대 (2016~2020) 제378회,NaN,NaN,NaN
2024994,처,법률용어 정비를 위한 정보위원회 소관 2개 법률 일부개정을 위한 법률안,위원장,2020-05-20,2020-05-20,원안가결,제20대 (2016~2020) 제378회,NaN,정보위원회,NaN
2024993,처,진실·화해를 위한 과거사정리 기본법 일부개정법률안(대안),위원장,2020-05-20,2020-05-20,원안가결,제20대 (2016~2020) 제378회,NaN,행정안전위원회,NaN
2024992,처,"화재예방, 소방시설 설치·유지 및 안전관리에 관한 법률 일부개정법률안(대안)",위원장,2020-05-20,2020-05-20,원안가결,제20대 (2016~2020) 제378회,NaN,행정안전위원회,NaN
2024990,처,재난 및 안전관리 기본법 일부개정법률안(대안),위원장,2020-05-20,2020-05-20,원안가결,제20대 (2016~2020) 제378회,NaN,행정안전위원회,NaN


In [10]:
bill_df = bill_df.fillna('')

In [11]:
bill_df.isnull().sum()

처리여부     0
의안명      0
제안자구분    0
제안일자     0
의결일자     0
의결결과     0
제안회기     0
제안이유     0
소관위원회    0
제안자      0
dtype: int64

### 제안이유 내용만 남기기

In [12]:
def clean_reason(df_col): 
    text = df_col[2:-2].replace('제안이유 및 주요내용','').replace('제안이유','')
    return re.sub('[-=.#/:$}·,■?]', ' ', text)

In [13]:
%time bill_df['제안이유'] = bill_df['제안이유'].apply(lambda x: clean_reason(x))

CPU times: user 374 ms, sys: 22 ms, total: 396 ms
Wall time: 617 ms


In [14]:
bill_df.head()

,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,제안회기,제안이유,소관위원회,제안자
의안번호,,,,,,,,,,
2007178,처,국가표준기본법 일부개정법률안(문미옥의원 등 28인),의원,2017-06-02,2017-11-24,수정가결,제20대 (2016~2020) 제351회,4차 산업혁명 시대의 도래와 함께 전세계적으로 참조표준 데이터와 다양한 기술의 융합...,산업통상자원중소벤처기업위원회,"['문미옥(더불어민주당/文美玉)', '강창일(더불어민주당/姜昌一)', '기동민(더불..."
2007177,처,동물보호법 일부개정법률안(한정애의원 등 12인),의원,2017-06-02,2017-12-16,폐기,제20대 (2016~2020) 제351회,현행법은 피학대동물에 대하여 치료 보호조치와 학대행위자로부터의 격리조치 등을 규...,농림축산식품해양수산위원회,"['한정애(더불어민주당/韓貞愛)', '강병원(더불어민주당/姜炳遠)', '박남춘(더불..."
2007176,처,실험동물에 관한 법률 일부개정법률안(한정애의원 등 12인),의원,2017-06-02,2017-12-01,대안반영폐기,제20대 (2016~2020) 제351회,최근 동물실험의 유효성 및 윤리성에 대한 관심의 증가로 세계 주요 국가들에서는 동물...,보건복지위원회,"['한정애(더불어민주당/韓貞愛)', '강병원(더불어민주당/姜炳遠)', '박남춘(더불..."
2007175,처,형사소송법 일부개정법률안(백혜련의원 등 12인),의원,2017-06-01,2020-05-29,임기만료폐기,제20대 (2016~2020) 제351회,국정농단 사건의 핵심 피의자인 정유라가 국외도피 245일만에 송환되어 경유지인 ...,법제사법위원회,"['백혜련(더불어민주당/白惠蓮)', '김상희(더불어민주당/金相姬)', '김영진(더불..."
2007174,처,산림자원의 조성 및 관리에 관한 법률 일부개정법률안(김철민의원 등 12인),의원,2017-06-01,2017-09-28,대안반영폐기,제20대 (2016~2020) 제351회,전세계적인 기후변화와 환경문제로 산림의 보호와 효과적인 관리 보존에 대한 관심이 증...,농림축산식품해양수산위원회,"['김철민(더불어민주당/金哲玟)', '강훈식(더불어민주당/姜勳植)', '김종회(국민..."


In [32]:
bill_df['제안자구분'].unique()

array(['의원', '정부', '의장', '위원장', '기타', '대통령'], dtype=object)

In [40]:
# 의원, 의장, 위원장
bill_df[bill_df['제안자구분'] == '위원장']

,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,제안회기,제안이유,소관위원회,제안자
의안번호,,,,,,,,,,
2000190,처,국회상임위원회 위원정수에 관한 규칙 일부개정규칙안,위원장,2016-06-13,2016-06-13,원안가결,제20대 (2016~2020) 제343회,,국회상임위원회 위원정수에 관한 규칙 개정 특별위원회,[]
2000689,처,가습기살균제 사고 진상규명과 피해구제 및 재발방지 대책마련을 위한 국정조사계획서...,위원장,2016-07-06,2016-07-06,원안가결,제20대 (2016~2020) 제343회,,2016-07-06,[]
2000690,처,민생경제 특별위원회 구성결의안,위원장,2016-07-06,2016-07-06,원안가결,제20대 (2016~2020) 제343회,,국회운영위원회,[]
2000691,처,미래일자리 특별위원회 구성결의안,위원장,2016-07-06,2016-07-06,원안가결,제20대 (2016~2020) 제343회,,국회운영위원회,[]
2000692,처,정치발전 특별위원회 구성결의안,위원장,2016-07-06,2016-07-06,원안가결,제20대 (2016~2020) 제343회,,국회운영위원회,[]
...,...,...,...,...,...,...,...,...,...,...
2024990,처,재난 및 안전관리 기본법 일부개정법률안(대안),위원장,2020-05-20,2020-05-20,원안가결,제20대 (2016~2020) 제378회,,행정안전위원회,
2024991,처,행정사법 일부개정법률안(대안),위원장,2020-05-20,2020-05-20,원안가결,제20대 (2016~2020) 제378회,복잡하고 다양한 행정 수요 증가에 대응하여 전문적인 행정사 서비스를 제공하고 행정사...,행정안전위원회,
2024992,처,"화재예방, 소방시설 설치·유지 및 안전관리에 관한 법률 일부개정법률안(대안)",위원장,2020-05-20,2020-05-20,원안가결,제20대 (2016~2020) 제378회,,행정안전위원회,


### 제안자 list화

In [15]:
# ? 제거 
bill_df['제안자'][19819] = bill_df['제안자'][19819].replace('?', '')
bill_df['제안자'][19819] = bill_df['제안자'][19821].replace('?', '')

In [16]:
def str_to_list(x): 
    try: 
        bill_pro = literal_eval(x)
    except: 
        bill_pro = ''
    return bill_pro

In [17]:
%time bill_df['제안자'] = bill_df['제안자'].apply(lambda x: str_to_list(x))

CPU times: user 964 ms, sys: 50 ms, total: 1.01 s
Wall time: 1.26 s


In [18]:
bill_df.head()

,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,제안회기,제안이유,소관위원회,제안자
의안번호,,,,,,,,,,
2007178,처,국가표준기본법 일부개정법률안(문미옥의원 등 28인),의원,2017-06-02,2017-11-24,수정가결,제20대 (2016~2020) 제351회,4차 산업혁명 시대의 도래와 함께 전세계적으로 참조표준 데이터와 다양한 기술의 융합...,산업통상자원중소벤처기업위원회,"[문미옥(더불어민주당/文美玉), 강창일(더불어민주당/姜昌一), 기동민(더불어민주당/..."
2007177,처,동물보호법 일부개정법률안(한정애의원 등 12인),의원,2017-06-02,2017-12-16,폐기,제20대 (2016~2020) 제351회,현행법은 피학대동물에 대하여 치료 보호조치와 학대행위자로부터의 격리조치 등을 규...,농림축산식품해양수산위원회,"[한정애(더불어민주당/韓貞愛), 강병원(더불어민주당/姜炳遠), 박남춘(더불어민주당/..."
2007176,처,실험동물에 관한 법률 일부개정법률안(한정애의원 등 12인),의원,2017-06-02,2017-12-01,대안반영폐기,제20대 (2016~2020) 제351회,최근 동물실험의 유효성 및 윤리성에 대한 관심의 증가로 세계 주요 국가들에서는 동물...,보건복지위원회,"[한정애(더불어민주당/韓貞愛), 강병원(더불어민주당/姜炳遠), 박남춘(더불어민주당/..."
2007175,처,형사소송법 일부개정법률안(백혜련의원 등 12인),의원,2017-06-01,2020-05-29,임기만료폐기,제20대 (2016~2020) 제351회,국정농단 사건의 핵심 피의자인 정유라가 국외도피 245일만에 송환되어 경유지인 ...,법제사법위원회,"[백혜련(더불어민주당/白惠蓮), 김상희(더불어민주당/金相姬), 김영진(더불어민주당/..."
2007174,처,산림자원의 조성 및 관리에 관한 법률 일부개정법률안(김철민의원 등 12인),의원,2017-06-01,2017-09-28,대안반영폐기,제20대 (2016~2020) 제351회,전세계적인 기후변화와 환경문제로 산림의 보호와 효과적인 관리 보존에 대한 관심이 증...,농림축산식품해양수산위원회,"[김철민(더불어민주당/金哲玟), 강훈식(더불어민주당/姜勳植), 김종회(국민의당/金鍾..."


In [19]:
bill_df.to_csv('bill-info-revised.csv', encoding='utf-8')

## EDA


In [20]:
bill_df = bill_df.sort_values('의안번호')
bill_df.head()

,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,제안회기,제안이유,소관위원회,제안자
의안번호,,,,,,,,,,
2000001,처,통일경제파주특별자치시의 설치 및 파주평화경제특별구역의 조성·운영과 지원에 관한 ...,의원,2016-05-30,2020-05-29,임기만료폐기,제20대 (2016~2020) 제342회,소득 양극화와 계층분화 심화ㆍ가계부채 심화ㆍ국가부채 급증ㆍ사회갈등 심화ㆍ일자리 부족...,외교통일위원회,"[박정(더불어민주당/朴釘), 김경협(더불어민주당/金炅俠), 노웅래(더불어민주당/盧雄..."
2000002,처,빅데이터의 이용 및 산업진흥 등에 관한 법률안(배덕광의원 등 14인),의원,2016-05-30,2020-05-29,임기만료폐기,제20대 (2016~2020) 제342회,빅데이터산업은 사물인터넷 클라우드 컴퓨팅 산업 등과 함께 정보통신산업의 성장을 ...,과학기술정보방송통신위원회,"[배덕광(새누리당/裵德光), 김세연(새누리당/金世淵), 김정훈(새누리당/金正薰), ..."
2000003,처,근로기준법 일부개정법률안(이찬열의원 등 11인),의원,2016-05-30,2020-05-29,임기만료폐기,제20대 (2016~2020) 제342회,지난 2015년 12월 고용노동부에서 발간한 ‘우리나라 노동시장의 모습’을 살펴보면...,환경노동위원회,"[이찬열(더불어민주당/李燦烈), 김경협(더불어민주당/金炅俠), 김현미(더불어민주당/..."
2000004,처,정부조직법 일부개정법률안(이종배의원 등 11인),의원,2016-05-30,2020-05-29,임기만료폐기,제20대 (2016~2020) 제342회,통계청에 따르면 우리나라는 2000년에 65세 이상의 노인인구가 전체인구의 7%로 ...,행정안전위원회,"[이종배(새누리당/李鍾培), 권성동(새누리당/權性東), 김재경(새누리당/金在庚), ..."
2000005,처,교육기본법 일부개정법률안(이찬열의원 등 10인),의원,2016-05-30,2020-05-29,임기만료폐기,제20대 (2016~2020) 제342회,현행법은 국가와 지방자치단체가 경제적 이유로 교육을 받기 곤란한 자를 위하여 장학제...,교육위원회,"[이찬열(더불어민주당/李燦烈), 김경협(더불어민주당/金炅俠), 백재현(더불어민주당/..."


In [21]:
bill_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25222 entries, 2000001 to ZZ23959
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   처리여부    25222 non-null  object
 1   의안명     25222 non-null  object
 2   제안자구분   25222 non-null  object
 3   제안일자    25222 non-null  object
 4   의결일자    25222 non-null  object
 5   의결결과    25222 non-null  object
 6   제안회기    25222 non-null  object
 7   제안이유    25222 non-null  object
 8   소관위원회   25222 non-null  object
 9   제안자     25222 non-null  object
dtypes: object(10)
memory usage: 2.1+ MB


총 25222개의 의안보고서가 존재함. 해당 의안보고서 중 실제 입법 활동과 관련된 보고서만 추릴 것. 의안번호가 ZZ로 시작하는 것 제외하고 추릴 것.

In [22]:
bill_df2 = bill_df.iloc[:24996, :]
bill_df2.head()

,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,제안회기,제안이유,소관위원회,제안자
의안번호,,,,,,,,,,
2000001,처,통일경제파주특별자치시의 설치 및 파주평화경제특별구역의 조성·운영과 지원에 관한 ...,의원,2016-05-30,2020-05-29,임기만료폐기,제20대 (2016~2020) 제342회,소득 양극화와 계층분화 심화ㆍ가계부채 심화ㆍ국가부채 급증ㆍ사회갈등 심화ㆍ일자리 부족...,외교통일위원회,"[박정(더불어민주당/朴釘), 김경협(더불어민주당/金炅俠), 노웅래(더불어민주당/盧雄..."
2000002,처,빅데이터의 이용 및 산업진흥 등에 관한 법률안(배덕광의원 등 14인),의원,2016-05-30,2020-05-29,임기만료폐기,제20대 (2016~2020) 제342회,빅데이터산업은 사물인터넷 클라우드 컴퓨팅 산업 등과 함께 정보통신산업의 성장을 ...,과학기술정보방송통신위원회,"[배덕광(새누리당/裵德光), 김세연(새누리당/金世淵), 김정훈(새누리당/金正薰), ..."
2000003,처,근로기준법 일부개정법률안(이찬열의원 등 11인),의원,2016-05-30,2020-05-29,임기만료폐기,제20대 (2016~2020) 제342회,지난 2015년 12월 고용노동부에서 발간한 ‘우리나라 노동시장의 모습’을 살펴보면...,환경노동위원회,"[이찬열(더불어민주당/李燦烈), 김경협(더불어민주당/金炅俠), 김현미(더불어민주당/..."
2000004,처,정부조직법 일부개정법률안(이종배의원 등 11인),의원,2016-05-30,2020-05-29,임기만료폐기,제20대 (2016~2020) 제342회,통계청에 따르면 우리나라는 2000년에 65세 이상의 노인인구가 전체인구의 7%로 ...,행정안전위원회,"[이종배(새누리당/李鍾培), 권성동(새누리당/權性東), 김재경(새누리당/金在庚), ..."
2000005,처,교육기본법 일부개정법률안(이찬열의원 등 10인),의원,2016-05-30,2020-05-29,임기만료폐기,제20대 (2016~2020) 제342회,현행법은 국가와 지방자치단체가 경제적 이유로 교육을 받기 곤란한 자를 위하여 장학제...,교육위원회,"[이찬열(더불어민주당/李燦烈), 김경협(더불어민주당/金炅俠), 백재현(더불어민주당/..."


### 의결결과 순으로 카테고리화

In [23]:
bill_df2['의결결과'].unique()

array(['임기만료폐기', '대안반영폐기', '수정가결', '원안가결', '철회', '수정안반영폐기', '폐기', '부결',
       '심사대상제외', '가결'], dtype=object)

In [24]:
result = bill_df2['의결결과'].value_counts()
result

임기만료폐기     15216
대안반영폐기      5568
원안가결        2670
수정가결        1123
철회           225
폐기           144
수정안반영폐기       41
부결             6
심사대상제외         2
가결             1
Name: 의결결과, dtype: int64

여기서 더 어떤 분석을 더 하면 좋을까? 어떻게 구분하는 것이 좋을까? 
- 가결되는 것의 특징 혹은 부결/폐기 되는 것을 특징을 알고 싶음.

### 의안이 처리되는데 소요되는 시간

In [25]:
bill_df2['제안일자'] = pd.to_datetime(bill_df2['제안일자'])
bill_df2['의결일자'] = pd.to_datetime(bill_df2['의결일자'])
bill_df2['의결처리기간'] = bill_df2['의결일자'] - bill_df2['제안일자'] 

<ipython-input-25-13802ef751c6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bill_df2['제안일자'] = pd.to_datetime(bill_df2['제안일자'])
<ipython-input-25-13802ef751c6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bill_df2['의결일자'] = pd.to_datetime(bill_df2['의결일자'])
<ipython-input-25-13802ef751c6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

In [26]:
bill_df2.head()

,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,제안회기,제안이유,소관위원회,제안자,의결처리기간
의안번호,,,,,,,,,,,
2000001,처,통일경제파주특별자치시의 설치 및 파주평화경제특별구역의 조성·운영과 지원에 관한 ...,의원,2016-05-30,2020-05-29,임기만료폐기,제20대 (2016~2020) 제342회,소득 양극화와 계층분화 심화ㆍ가계부채 심화ㆍ국가부채 급증ㆍ사회갈등 심화ㆍ일자리 부족...,외교통일위원회,"[박정(더불어민주당/朴釘), 김경협(더불어민주당/金炅俠), 노웅래(더불어민주당/盧雄...",1460 days
2000002,처,빅데이터의 이용 및 산업진흥 등에 관한 법률안(배덕광의원 등 14인),의원,2016-05-30,2020-05-29,임기만료폐기,제20대 (2016~2020) 제342회,빅데이터산업은 사물인터넷 클라우드 컴퓨팅 산업 등과 함께 정보통신산업의 성장을 ...,과학기술정보방송통신위원회,"[배덕광(새누리당/裵德光), 김세연(새누리당/金世淵), 김정훈(새누리당/金正薰), ...",1460 days
2000003,처,근로기준법 일부개정법률안(이찬열의원 등 11인),의원,2016-05-30,2020-05-29,임기만료폐기,제20대 (2016~2020) 제342회,지난 2015년 12월 고용노동부에서 발간한 ‘우리나라 노동시장의 모습’을 살펴보면...,환경노동위원회,"[이찬열(더불어민주당/李燦烈), 김경협(더불어민주당/金炅俠), 김현미(더불어민주당/...",1460 days
2000004,처,정부조직법 일부개정법률안(이종배의원 등 11인),의원,2016-05-30,2020-05-29,임기만료폐기,제20대 (2016~2020) 제342회,통계청에 따르면 우리나라는 2000년에 65세 이상의 노인인구가 전체인구의 7%로 ...,행정안전위원회,"[이종배(새누리당/李鍾培), 권성동(새누리당/權性東), 김재경(새누리당/金在庚), ...",1460 days
2000005,처,교육기본법 일부개정법률안(이찬열의원 등 10인),의원,2016-05-30,2020-05-29,임기만료폐기,제20대 (2016~2020) 제342회,현행법은 국가와 지방자치단체가 경제적 이유로 교육을 받기 곤란한 자를 위하여 장학제...,교육위원회,"[이찬열(더불어민주당/李燦烈), 김경협(더불어민주당/金炅俠), 백재현(더불어민주당/...",1460 days


In [27]:
bill_df2.sort_values('의결처리기간', ascending=False).head(3)

,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,제안회기,제안이유,소관위원회,제안자,의결처리기간
의안번호,,,,,,,,,,,
2000001,처,통일경제파주특별자치시의 설치 및 파주평화경제특별구역의 조성·운영과 지원에 관한 ...,의원,2016-05-30,2020-05-29,임기만료폐기,제20대 (2016~2020) 제342회,소득 양극화와 계층분화 심화ㆍ가계부채 심화ㆍ국가부채 급증ㆍ사회갈등 심화ㆍ일자리 부족...,외교통일위원회,"[박정(더불어민주당/朴釘), 김경협(더불어민주당/金炅俠), 노웅래(더불어민주당/盧雄...",1460 days
2000019,처,국가재정법 일부개정법률안(박명재의원 등 14인),의원,2016-05-30,2020-05-29,임기만료폐기,제20대 (2016~2020) 제342회,경상북도 울릉도ㆍ독도 지역은 동해 유일의 접경지역으로서 군사적ㆍ국가 안보적 차원에서...,기획재정위원회,"[박명재(새누리당/朴明在), 강석호(새누리당/姜碩鎬), 경대수(새누리당/慶大秀), ...",1460 days
2000044,처,선박투자회사법 일부개정법률안(김성찬의원 등 10인),의원,2016-05-30,2020-05-29,임기만료폐기,제20대 (2016~2020) 제342회,선박투자회사를 현물출자 방법으로 설립할 수 있도록 허용하여 투자자들이 다양한 형태의...,농림축산식품해양수산위원회,"[김성찬(새누리당/金盛贊), 김재경(새누리당/金在庚), 박대출(새누리당/朴大出), ...",1460 days


In [28]:
days = pd.Series(bill_df2['의결처리기간'].unique()).sort_values(ascending=False)
days

0     1460 days
15    1459 days
18    1458 days
22    1457 days
28    1456 days
         ...   
99       4 days
493      3 days
668      2 days
111      1 days
52       0 days
Length: 1300, dtype: timedelta64[ns]

In [29]:
days.mean()

Timedelta('683 days 01:47:26.769230')

In [30]:
# 소관위원회만 추렸는데 왜 숫자가 나오지? 
bill_df2['소관위원회'].unique()

array(['외교통일위원회', '과학기술정보방송통신위원회', '환경노동위원회', '행정안전위원회', '교육위원회',
       '법제사법위원회', '기획재정위원회', '농림축산식품해양수산위원회', '산업통상자원위원회', '안전행정위원회',
       '산업통상자원중소벤처기업위원회', '정무위원회', '정보위원회', '국회운영위원회', '국토교통위원회',
       '보건복지위원회', '교육문화체육관광위원회', '문화체육관광위원회', '미래창조과학방송통신위원회', '여성가족위원회',
       '2016-06-09', '국방위원회', '국회상임위원회 위원정수에 관한 규칙 개정 특별위원회',
       '2016-06-20', '2016-06-21', '2016-06-22', '2016-07-06', '윤리특별위원회',
       '2016-07-27', '대법관(김재형) 임명동의에 관한 인사청문특별위원회', '2016-09-02',
       '예산결산특별위원회', '', '2016-09-07', '2016-09-24',
       '평창동계올림픽 및 국제경기대회지원 특별위원회', '저출산·고령화대책 특별위원회', '2016-11-03',
       '2016-11-17', '2016-12-09', '2016-12-03', '2016-12-08',
       '2016-12-20', '2016-12-29', '2017-01-20', '2017-02-01',
       '2017-02-03', '2017-02-06', '2017-02-07', '2017-02-23',
       '2017-03-02', '2017-03-16', '2017-03-28',
       '국무총리(이낙연) 임명동의에 관한 인사청문특별위원회', '헌법재판소장(김이수) 임명동의에 관한 인사청문특별위원회',
       '2017-05-29', '2017-06-12', '2017-06-22',
       '대법관(박정화·조재연) 임명동의에 관한 인사청문특별위원회', 